In [57]:
from gurobipy import *
from math import factorial
import pandas as pd
import time
import numpy as np
from matplotlib import pyplot as plt

In [58]:
coord = {}
for i in range(1,189):
    if i < 33:
        coord[i] = [1,i]
    elif i < 65:
        coord[i] = [2,i-32]
    elif i < 96:
        coord[i] = [3,i-64]
    elif i < 127:
        coord[i] = [5,i-95]
    elif i < 158:
        coord[i] = [6,i-126]
    elif i < 189:
        coord[i] = [7,i-157]
d = {}
for i in range(1,189):
    for j in range(1,189):
        d[i,j] = abs(coord[i][0] - coord[j][0]) + abs(coord[i][1]-coord[j][1])

In [59]:
df = pd.read_excel (r'BD SEAT .xlsx')
df2 = df.dropna()
lim = 10

values = df2['UnitDesignator'].value_counts()[0:lim]
courses = values.keys()
#plt.bar(courses, values, color = 'darkblue')
#plt.xlabel("Seats")
#plt.ylabel("Number of times purchased")
#plt.title("Top 10 most purchased seats")
#plt.show()
maximo = max(df2['UnitDesignator'].value_counts())
normalizado = df2['UnitDesignator'].value_counts()/maximo


#plt.bar(normalizado.index[0:10],normalizado[0:10], color = 'darkblue')
#plt.xlabel("Seats")
#plt.ylabel("Seat rank")
#plt.title("Top 10 most purchased seats")
#plt.show()


vec = {}
for i in range(0, len(normalizado.index)):
    if len(normalizado.index[i]) == 2 :
        if normalizado.index[i][-1] == 'A':
            vec[int(normalizado.index[i][0])] = 1
        elif normalizado.index[i][-1] == 'B':
            vec[int(normalizado.index[i][0])+32] = 1
        elif normalizado.index[i][-1] == 'C':
            vec[int(normalizado.index[i][0])+32+32] = 1
        elif normalizado.index[i][-1] == 'D':
            vec[int(normalizado.index[i][0])+32+32+31] = 1
        elif normalizado.index[i][-1] == 'E':
            vec[int(normalizado.index[i][0])+32+32+31+31] = 1
        elif normalizado.index[i][-1] == 'F':
            vec[int(normalizado.index[i][0])+32+32+31+31+31] = 1
            
    elif len(normalizado.index[i]) == 3 :
        if normalizado.index[i][-1] == 'A':
            vec[int(normalizado.index[i][0:2])] = 1
        elif normalizado.index[i][-1] == 'B':
            vec[int(normalizado.index[i][0:2])+32] = 1
        elif normalizado.index[i][-1] == 'C':
            vec[int(normalizado.index[i][0:2])+32+32] = 1
        elif normalizado.index[i][-1] == 'D':
            vec[int(normalizado.index[i][0:2])+32+32+31] = 1
        elif normalizado.index[i][-1] == 'E':
            vec[int(normalizado.index[i][0:2])+32+32+31+31] = 1
        elif normalizado.index[i][-1] == 'F':
            vec[int(normalizado.index[i][0:2])+32+32+31+31+31] = 1
            
            
contador = -1
for i in vec.keys():
    contador += 1
    vec[i] = normalizado.values[contador]
    
beta = {}
delta = 6 
    
df3 = df[df['DepartureDate'] == '2022-06-08']
df4 = df3[df3['FlightNumber'] == 59421]
qReal = df4['RecordLocator'].value_counts()

In [61]:
depDates = df['DepartureDate'].unique()



flightCounter = 0
for iii in depDates[3:]:

    dff3 = df[df['DepartureDate'] == iii]
    NumberFlights = dff3['FlightNumber'].unique()
    for jjj in NumberFlights:
        dff4 = dff3[dff3['FlightNumber'] == jjj]
        qReal = dff4['RecordLocator'].value_counts()
        flightCounter += 1
        
        
        
        I = range(1,189) #188 sillas entones el range(1,189)

        c = {}
        a = {}
        for i in I:
            a[i] = 1
            #c[i] = 1
        vecc = [39,34,34,34,34,27,27,27,27,27,27,29,29,18,18,18,18,18,18,18,18,18,18,14,14,14,14,14,14,14,14,14]
        vec2 = [34,29,29,29,29,22,22,22,22,22,22,24,24,12,12,12,12,12,12,12,12,12,12,9,9,9,9,9,9,9,9,9]


        for i in range(0,32):
            c[i+1] = vecc[i]
            c[i+33] = vec2[i]
            if i != 31:
                c[i+65] = vecc[i]
                c[i+96] = vecc[i]
                c[i+127] = vec2[i]
                c[i+158] = vecc[i]


        costoReal = c.copy()

        bono = 100
        for i in c:
            if i != 32:
                c[i] = c[i] + bono*vec[i]

        c[32] = 500
        c[64] = 500
        c[31] = 500
        c[63] = 500
        c[95] = 500
        c[126] = 500
        c[157] = 500
        c[188] = 500
        c[31] = 500
        c[62] = 500
        c[94] = 500
        c[125] = 500   
        c[156] = 500
        c[187] = 500  


        q = 3
        qR = []
        for i in qReal:
            qR.append(i)



        if q > 1:
            fact = math.factorial(q)/(math.factorial(q-2))#*math.factorial(2))
        else:
            fact = 0


        
        sortedC = dict(sorted(c.items(), key=lambda x:x[1]))

        cntr = 0
        indexes = []
        for i in reversed(sortedC):
            if cntr <= len(c)*0.8-1:
                a[i] = 0
                indexes.append(i)
            cntr += 1
        indexes = list(reversed(indexes))
        
        
        #### Empieza a optimizar
        
        seats = {}
        tInicial = time.time()





        timee = {}
        contBeta = 0
        t = 0
        OFS = {}
        superCounter = 0
        while sum(a.values()) > 0:
            tIter = time.time()
            #print(sum(a.values()))
            
            if len(qR) - t <= 0:
                break
            
            t = t + 1
            while qR[t-1] >= 20:
                t = t + 1
                
            q = qR[t-1]
            if q > 1:
                fact = math.factorial(q)/(math.factorial(q-2))#*math.factorial(2))
            else:
                fact = 0
            
            print(q)


            m = Model("Proyecto: "+str(t))

            # Variable de decisión
            x = {(i):m.addVar(vtype=GRB.BINARY, obj = 1.8*c[i], name="x"+str(i)) for i in I}
            y = {(i,j):m.addVar(vtype=GRB.BINARY, obj = -1.5*d[i,j], name="y"+str(i)+str(j)) for i in I for j in I if j!=i}



            # Restricciones
            for i in I:
                #(1c)
                m.addConstr(x[i] <= 1)
                #(1d)
                m.addConstr(x[i] <= a[i])
                # propios
                m.addConstr(x[i]*(q-1) == quicksum(y[i,m] for m in I if m!=i))
                m.addConstr(x[i]*(q-1) == quicksum(y[m,i] for m in I if m!=i))






            m.addConstr(quicksum(x[i] for i in I) == q)

                #(1i)
            m.addConstr(quicksum(y[j,i] for j in I for i in I if j!=i) == fact)




            m.ModelSense = 1 # Para minimizar

            m.setParam("OutputFlag",0)
            m.setParam('Heuristics', 0)
            m.setParam('Threads', 8)
            m.setParam("DualReductions",0)
            m.setParam("InfUnbdInfo", 1)


            if q >= 10 and contBeta == 0:

                m.setParam('SolutionLimit', 1)
                m.setParam('MIPGap', 0.2)
                #m.setParam("TimeLimit", 1000)

                delta = 2
                for i in range(1,189):
                    for j in range(1,189):
                        if d[i,j] >= delta:
                            beta[i,j] = 1
                        else:
                            beta[i,j] = 0  

            elif q < 10 and contBeta == 0 and q >= 7:  

                m.setParam('MIPGap', 0.15)
                m.setParam('NodeLimit', 500000)
                delta = 7
                for i in range(1,189):
                    for j in range(1,189):
                        if d[i,j] >= delta:
                            beta[i,j] = 1
                        else:
                            beta[i,j] = 0  

            elif q < 7 and contBeta == 0:

                m.setParam('MIPGap', 0.10)
                m.setParam('NodeLimit', 100000)  

                delta = 7
                for i in range(1,189):
                    for j in range(1,189):
                        if d[i,j] >= delta:
                            beta[i,j] = 1
                        else:
                            beta[i,j] = 0  

            for i in I:
                    for j in I:
                        if i!=j:
                            m.addConstr(y[i,j] <= beta[i,j])

            m.update()
            print('Llego!!')
            m.optimize()


            if m.Status == 3 and delta >= 1: 

                t = t - 1
                contBeta += 1
                delta = delta - 1
                for i in range(1,189):
                    for j in range(1,189):
                        if d[i,j] >= delta:
                            beta[i,j] = 1
                        else:
                            beta[i,j] = 0
            elif m.Status == 3 and delta <= 0:
                break
            else:

                #if t%20==0:# and q!=1:
                #print("Iteration: " + str(t))
                    #timee.append((time.time()-tIter)/60)
                timee[t] = round((time.time()-tIter)/60,4)
                OFS[t] = m.objVal
                #print('Time for iteration: ' + str(timee[t]))
                #OF = m.objVal


                vals = { k : v.X for k,v in x.items() }
                for b in vals:
                    if vals[b] == 1:
                        a[b] = 0
                        seats[b] = t

                if superCounter <= len(indexes):
                    for i in indexes[superCounter:superCounter + q]:
                        a[i] = 1


                    superCounter += q





        tTotal = round(time.time() - tInicial,5) 

    
    
    
    
        results = pd.DataFrame(index = list(range(1,33)),columns = ['A','B','C','D','E','F'])

        for i in seats.keys():
            if i <= 32:
                results['A'][i] = seats[i]
            elif i <= 64:
                results['B'][i-32] = seats[i]
            elif i <= 95:
                results['C'][i-64] = seats[i]
            elif i <= 126:
                results['D'][i-95] = seats[i]
            elif i <= 157:
                results['E'][i-126] = seats[i]
            elif i <= 188:
                results['F'][i-157] = seats[i]

        results.to_excel(r'C:\Users\Germa\OneDrive - Universidad de los andes\Libros U\2022-20 Flujo en Redes\Proyecto\Fase 2\resultados'+str(flightCounter+44)+'.xlsx')












7
Llego!!
7
Llego!!
6
Llego!!
5
Llego!!
4
Llego!!
4
Llego!!
4
Llego!!
4
Llego!!
4
Llego!!
3
Llego!!
3
Llego!!
3
Llego!!
3
Llego!!
3
Llego!!
3
Llego!!
3
Llego!!
3
Llego!!
3
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
2
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!
1
Llego!!


KeyboardInterrupt: 

In [62]:
iii

numpy.datetime64('2022-06-27T00:00:00.000000000')

In [63]:
jjj

55281

In [40]:
iii

numpy.datetime64('2022-06-23T00:00:00.000000000')

In [41]:
jjj

83381

In [39]:
startiii = np.datetime64('2022-06-15T00:00:00.000000000')
startjjj = 55361